# Imports

In [12]:
# Watch for any changes in model.py, and re-load it automatically.
% load_ext autoreload
% autoreload 2

import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
!pip install nltk
import nltk
nltk.download('punkt')
import numpy as np
import torch
import torch.utils.data as data

from data_loader import get_loader
from torchvision import transforms
from model import EncoderCNN, DecoderRNN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# CUDA
In the next code cell we define a device that you will use move PyTorch tensors to GPU (if CUDA is available). Run this code cell before continuing.

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Loader

In [5]:
# Define a transform to pre-process the training images.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Set the minimum word count threshold.
vocab_threshold = 5

# Specify the batch size.
batch_size = 10

# Obtain the data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=True)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...



  0%|          | 532/414113 [00:00<01:17, 5319.29it/s]

Done (t=0.87s)
creating index...
index created!
Obtaining caption lengths...



 21%|██        | 86186/414113 [00:14<00:52, 6271.68it/s]


 42%|████▏     | 173118/414113 [00:27<00:55, 4304.32it/s]


 63%|██████▎   | 258920/414113 [00:41<00:24, 6272.44it/s]


 83%|████████▎ | 344108/414113 [00:55<00:11, 6362.73it/s]


100%|██████████| 414113/414113 [01:06<00:00, 6242.83it/s]

# Batch
Run the code cell below to generate a batch. The get_train_indices method in the CoCoDataset class first samples a caption length, and then samples batch_size indices corresponding to training data points with captions of that length. These indices are stored below in indices.
These indices are supplied to the data loader, which then is used to retrieve the corresponding data points. The pre-processed images and captions in the batch are stored in images and captions.

In [6]:
# Randomly sample a caption length, and sample indices with that length.
indices = data_loader.dataset.get_train_indices()
print('sampled indices:', indices)

# Create and assign a batch sampler to retrieve a batch with the sampled indices.
new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
data_loader.batch_sampler.sampler = new_sampler
    
# Obtain the batch.
images, captions = next(iter(data_loader))
    
print('images.shape:', images.shape)
print('captions.shape:', captions.shape)

sampled indices: [330135, 220168, 149566, 180988, 125178, 25963, 99629, 317992, 358618, 20330]
images.shape: torch.Size([10, 3, 224, 224])
captions.shape: torch.Size([10, 11])


# Encoder

In [7]:
# Specify the dimensionality of the image embedding.
embed_size = 256

# Initialize the encoder. (Optional: Add additional arguments if necessary.)
encoder = EncoderCNN(embed_size)

# Move the encoder to GPU if CUDA is available.
encoder.to(device)
    
# Move last batch of images (from Step 2) to GPU if CUDA is available.   
images = images.to(device)

# Pass the images through the encoder.
features = encoder(images)

print('type(features):', type(features))
print('features.shape:', features.shape)

# Check that your encoder satisfies some requirements of the project! :D
assert type(features)==torch.Tensor, "Encoder output needs to be a PyTorch Tensor." 
assert (features.shape[0]==batch_size) & (features.shape[1]==embed_size), "The shape of the encoder output is incorrect."

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth

100%|██████████| 102502400/102502400 [00:01<00:00, 53024084.88it/s]

type(features): <class 'torch.Tensor'>
features.shape: torch.Size([10, 256])


In [9]:
print(captions.shape)
print(captions[:, :].shape)

torch.Size([10, 11])
torch.Size([10, 11])


# Decoder

In [13]:
# Specify the number of features in the hidden state of the RNN decoder.
hidden_size = 256

# Store the size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the decoder.
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move the decoder to GPU if CUDA is available.
decoder.to(device)
    
# Move last batch of captions (from Step 1) to GPU if CUDA is available 
captions = captions.to(device)

# Pass the encoder output and captions through the decoder.
outputs = decoder.forward(features, captions)

print('type(outputs):', type(outputs))
print('outputs.shape:', outputs.shape)

# Check that your decoder satisfies some requirements of the project! :D
assert type(outputs)==torch.Tensor, "Decoder output needs to be a PyTorch Tensor."
assert (outputs.shape[0]==batch_size) & (outputs.shape[1]==captions.shape[1]) & (outputs.shape[2]==vocab_size), "The shape of the decoder output is incorrect."

type(outputs): <class 'torch.Tensor'>
outputs.shape: torch.Size([10, 11, 8855])
